# Libraries

In [1]:
from openpyxl import load_workbook
import pandas as pd
import os

# Params

In [2]:
%%time
week = '11'
anio = 2021
path = '../../../../../../../02Paises/Colombia/MULTIDROGAS/2021/03 Marzo/S' + week
filename = '/2021_' + week + '_Multidrogas.xlsx'

wb = load_workbook(path + filename)

Wall time: 197 ms


# Import data

## Stock

Carga de ambas hojas de ineventario

In [3]:
df_stock = pd.DataFrame()
for sheet_stock in [s for s in wb.sheetnames if s.lower().startswith('inv')]:
    if 'car' in sheet_stock.lower():
        df_aux = pd.read_excel(path + filename, sheet_name=sheet_stock)
        df_aux['CodSuc'] = 1001
    elif 'mon' in sheet_stock.lower():
        df_aux = pd.read_excel(path + filename, sheet_name=sheet_stock)
        df_aux['CodSuc'] = 1003
    else:
        print('Sheet not found!')
        break
    df_stock = pd.concat([df_stock, df_aux], axis=0)
df_stock.shape

(333, 8)

## Sell Out

Carga de ambas hojas de ventas

In [4]:
df_sellout = pd.DataFrame()
for sheet_so in [s for s in wb.sheetnames if s.lower().startswith('rot')]:
    if 'car' in sheet_so.lower():
        df_aux = pd.read_excel(path + filename, sheet_name=sheet_so)
        df_aux['CodSuc'] = 1001
    elif 'mon' in sheet_so.lower():
        df_aux = pd.read_excel(path + filename, sheet_name=sheet_so)
        df_aux['CodSuc'] = 1003
    else:
        print('Sheet not found!')
        break
    df_sellout = pd.concat([df_sellout, df_aux], axis=0)
df_sellout.shape

(819, 15)

# Transformation

Se filtran las columnas de interés:

In [5]:
stock_filt = df_stock.pivot_table(index=['Material', 'Texto breve de material', 'CodSuc'], values=['Libre utilización'], aggfunc='sum').reset_index()

sellout_filt = df_sellout.pivot_table(index=['Material', 'Denominación', 'CodSuc'], values=['Ctd.facturada'], aggfunc='sum').reset_index()

Se crea la columna ID y se cruzan los datos:

In [6]:
stock_filt['ID'] = stock_filt['Material'].astype(str) + stock_filt['CodSuc'].astype(str)
sellout_filt['ID'] = sellout_filt['Material'].astype(str) + sellout_filt['CodSuc'].astype(str)

so_stock = pd.merge(sellout_filt, stock_filt, on='ID', how='outer')

In [7]:
print(stock_filt['Libre utilización'].sum(), sellout_filt['Ctd.facturada'].sum())

33238 2282


Función para "rellenar" el valor entre 2 columnas:

In [8]:
def fill_value(first_value, second_value, value_replace=0):
    '''
    first_value: value to check if it's null.
    first_value: value to check if it's null.
    value_replace: if both values are null, we'll fill them with value of value_replace.
    '''
    if pd.isnull(first_value):
        return second_value
    elif pd.isnull(second_value):
        return first_value
    elif pd.notnull(first_value):
        return first_value
    else:
        return value_replace

Limpieza y creación de columnas:

In [9]:
%%time
so_stock['cod sku Cliente'] = so_stock['Material_y'].combine(so_stock['Material_x'], fill_value)
so_stock['CodSuc'] = so_stock['CodSuc_y'].combine(so_stock['CodSuc_x'], fill_value)

#so_stock['nombre SKU'] = so_stock['Denominación'].combine(so_stock['Texto breve de material'], fill_value)
so_stock['nombre SKU'] = so_stock['Texto breve de material'].combine(so_stock['Denominación'], fill_value)
so_stock['AÑO'] = anio
so_stock['Sucursal'] = so_stock['CodSuc'].map({1001:'Cartagena', 1003:'Monteria'})
so_stock['Ven Und'] = so_stock['Ctd.facturada']
so_stock['Ven Und'].fillna(0, inplace=True)
so_stock['Inv Und'] = so_stock['Libre utilización']
so_stock['Inv Und'].fillna(0, inplace=True)
so_stock['SEMANA'] = int(week)
so_stock['Nit'] = 823000097
so_stock['Cliente'] = 'Multidrogas'
so_stock['Canal'] = 'Mayoristas'
so_stock['Cod Sap'] = 0
so_stock['ProPstID'] = 0
so_stock['Precio'] = 0
so_stock['cod sku Cliente'] = so_stock['cod sku Cliente'].astype(str)

so_stock.drop(['Material_x', 'Material_y', 'CodSuc_x', 'CodSuc_y', 'Ctd.facturada', 'Libre utilización', 'ID', 'Denominación', 'Texto breve de material'], axis=1, inplace=True)

#so_stock.rename({'Año':'AÑO', 'Material':'cod sku Cliente'}, axis=1, inplace=True)

Wall time: 42.1 ms


In [10]:
so_stock.isnull().sum()

cod sku Cliente    0
CodSuc             0
nombre SKU         0
AÑO                0
Sucursal           0
Ven Und            0
Inv Und            0
SEMANA             0
Nit                0
Cliente            0
Canal              0
Cod Sap            0
ProPstID           0
Precio             0
dtype: int64

In [11]:
so_stock = so_stock.pivot_table(index=['AÑO', 'SEMANA', 'Nit', 'Cliente', 'Canal', 'CodSuc', 'Sucursal', 'Cod Sap', 'cod sku Cliente', 'nombre SKU'], values=['Ven Und', 'Inv Und', 'ProPstID', 'Precio'], aggfunc='sum').reset_index()

so_stock = so_stock[['AÑO', 'SEMANA', 'Nit', 'Cliente', 'Canal', 'CodSuc', 'Sucursal', 'Cod Sap', 'cod sku Cliente', 'nombre SKU', 'Ven Und', 'Inv Und', 'ProPstID', 'Precio']].copy()

# Export

Exportación de datos final

so_stock.to_excel(path + '/MULTIDROGAS W2.xlsx', sheet_name='Sabana', index=False)